# Encriptacion de tarjeta de credito

In [12]:
from Crypto.Cipher import AES
from Crypto import Random
import codecs

credit_card_number = "4532294977918448"
print('Number to encrypt = ' + credit_card_number)

round_keys = []

def xor(a, b):
    a = '{:032b}'.format(int(a, 16))
    tmp = ''
    for i in range(len(a) - 5):
        tmp += str(int(a[i]) ^ int(b[i]))
    return tmp

def enc(card_number, rounds, produce_keys):

    encoded_number = '{0:054b}'.format(int(card_number))
    l = encoded_number[:27]
    r = encoded_number[27:]

    if produce_keys:
        for i in range(rounds):
            key = Random.new().read(16)
            key = codecs.encode(key,'hex_codec')
            round_keys.append(key)

    for i in range(rounds):        
        
        key = round_keys[i]
       
        obj = AES.new(codecs.decode(key,'hex_codec'),AES.MODE_EAX)
        hex_r = '{:08x}'.format(int(r + '0'*5, 2))
        b = hex_r + '0'*23 + str(i+1)
        enc_res = obj.encrypt(codecs.decode(b,'hex_codec'))
        enc_res = str(codecs.encode(enc_res,'hex_codec'))[:7] + '0'
        tmp = r
        r = xor(enc_res, l+'0'*5)
        l = tmp

    return (l , r)

def dec(cipher, rounds):

    encoded_number = '{0:054b}'.format(int(cipher))
    l = encoded_number[:27]
    r = encoded_number[27:]

    for i in range(rounds,0,-1):
        key = round_keys[i-1]
        obj = AES.new(key.decode('hex'))
        
        hex_l = '{:08x}'.format(int(l + '0'*5, 2))
        b = hex_l + '0'*23 + str(i)
        enc_res = obj.encrypt(b.decode('hex')).encode('hex')[:7] + '0'
        tmp = l
        l = xor(enc_res, r+'0'*5)
        r = tmp

    return (l , r)


l , r = enc(credit_card_number, 6, True)
final = l + r
final = int(final, 2)
while final > 9999999999999999:
    l , r = enc(final, 6, False)
    final = l + r
    final = int(final, 2)


l , r = dec(final, 6)
final = l + r
final = int(final, 2)
while final > 9999999999999999:
    l , r = dec(final, 6)
    final = l + r
    final = int(final, 2)

Number to encrypt = 4532294977918448


ValueError: invalid literal for int() with base 16: "b'7ccab0"

In [77]:
import re
import argparse

from Crypto.Cipher import AES
from bitstring import Bits
from random import randint
from sys import byteorder

cycles = 0


def rounds(right, left, key, roundnum, decrypt):
    """
    Does the repetitions of the Feistel Network.
    The value of decrypt dictates whether the Feistel is reversed
    or not. Reversed Feistel is used for the decryption only.
    """
    """ print("\n-----------------")
    print("Right Key: " + right.bin)
    print("Left Key:  " + left.bin)
    print("-----------------\n")
    print("Starting AES Rounds..") """

    if decrypt:
        for r in range(roundnum-1, -1, -1):
            # In order to decrypt, we need to reverse the order of the
            # AES input blocks, hence we need to reverse roundnum order.
            # Round is 101 bits as to make the total block size 128-bits
            #print("Decrypting Round: " + repr(r+1))
            round = Bits(uint = r, length = 101)
            #print("-----------------")

            temp = left
            left = aes_enc(left, key, round) ^ right
            right = temp
            #print("Right Key: " + right.bin)
            #print("Left Key:  " + left.bin)
            #print("-----------------")
    else:
        for r in range(0, roundnum):
            # Round is 101 bits as to make the total block size 128-bits
            #print("Round Number: " + repr(r+1))
            round = Bits(uint = r, length = 101)
            #print("-----------------")

            temp = right
            right = aes_enc(right, key, round) ^ left
            left = temp
            #print("Right Key: " + right.bin)
            #print("Left Key:  " + left.bin)
            #print("-----------------")

    # In order to display the number properly we need to pad it to 64 bits.
    # Otherwise, we will get a negative, wrong value
    pad = Bits(bin="0000000000")
    whole = pad + left + right

    return whole.int


def aes_enc(half, key, round):
    """
    This is the AES encryption used for each round in the Feistel network.
    We use ECB with no problem since we encrypt only one block with it.
    """

    encrypter = AES.new(key.encode("utf8"), AES.MODE_ECB)
    block = half + round
    output = encrypter.encrypt(block.bin.encode("utf8"))

    # The output is in hex bytes format, if we iterate through it we can
    # get the decimal value of every digit
    output = ''.join([str(x) for x in output])
    output = bin(int(output))
    output = output[2:29]

    return Bits(bin = output)


def mainloop(cardnum, key, roundnum, decrypt):
    """
    This loop keeps the Feistel going until we get a 16 digit number.
    """

    global cycles
    # This regex is used to display the number in groups of four digits
    card = re.findall('\d{4}', repr(cardnum))
    #if cycles == 0:
        #print("Card Number: " + ' '.join(num for num in card))
    # Turn the card number into a binary number with exactly 54 digits
    cardnum = Bits(uint = cardnum, length = 54)

    # Split the number into two 27-bit numbers
    left = cardnum[:27]
    right = cardnum[27:]

    while True:
        result = rounds(right, left, key, roundnum, decrypt)

        if len(repr(result)) > 16:
            # We need exactly 16 digits. If we have more, we start again
            #print("\nThe result is more than 16 digits, cycling...")
            cycles += 1
            result = Bits(uint = result, length = 54)

            left = result[:27]
            right = result[27:]
        else:
            result = format(result, "016d")
            output = re.findall('\d{4}', repr(result))
            #print("Result: " + ' '.join(num for num in output) + "\n")
            #print("Cycled " + repr(cycles) + " times!")
            return result




In [45]:
decrypt = False 
cardnum = randint(1000000000000000, 9999999999999999)
key = 32*"A"
roundnum = randint(3, 9)
print(cardnum,roundnum)
result = mainloop(cardnum, key, roundnum, decrypt)
print(result)

7342179663921640 6

-----------------
Right Key: 010101111101100000111101000
Left Key:  011010000101011010110001101
-----------------

Starting AES Rounds..
Round Number: 1
-----------------
Right Key: 110010111000110011010111100
Left Key:  010101111101100000111101000
-----------------
Round Number: 2
-----------------
Right Key: 100100011100011000010001101
Left Key:  110010111000110011010111100
-----------------
Round Number: 3
-----------------
Right Key: 000010011111011111100100001
Left Key:  100100011100011000010001101
-----------------
Round Number: 4
-----------------
Right Key: 010101001101100101001011101
Left Key:  000010011111011111100100001
-----------------
Round Number: 5
-----------------
Right Key: 111101000100111101011111100
Left Key:  010101001101100101001011101
-----------------
Round Number: 6
-----------------
Right Key: 101011010111100111000110101
Left Key:  111101000100111101011111100
-----------------

The result is more than 16 digits, cycling...

---------------

In [56]:
decrypt = True
cardnum = 1383032367418336
key = 32*"A"
roundnum = randint(3, 9) 
print(roundnum,key)
result = mainloop(cardnum, key, 6, decrypt)

7 AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

-----------------
Right Key: 000111110010000101111100000
Left Key:  000100111010011101110001010
-----------------

Starting AES Rounds..
Decrypting Round: 6
-----------------
Right Key: 000100111010011101110001010
Left Key:  100000010110011110011101110
-----------------
Decrypting Round: 5
-----------------
Right Key: 100000010110011110011101110
Left Key:  100100000110010010010111101
-----------------
Decrypting Round: 4
-----------------
Right Key: 100100000110010010010111101
Left Key:  001001011010101001110011100
-----------------
Decrypting Round: 3
-----------------
Right Key: 001001011010101001110011100
Left Key:  001010101100001100100100010
-----------------
Decrypting Round: 2
-----------------
Right Key: 001010101100001100100100010
Left Key:  101011010111100111000110101
-----------------
Decrypting Round: 1
-----------------
Right Key: 101011010111100111000110101
Left Key:  111101000100111101011111100
-----------------

The result is more tha

In [50]:
import pandas as pd 

df_cinta = pd.read_csv("cinta_fac.csv") 

print(df_cinta)

                  nombre                pan    valor moneda
0       Stella Howell Jr.  5053469667779484  1000000    COP
1          Tracy Schaefer  5086090068617676  2000000    COP
2              Joel Jones  5054876152300379  3000000    COP
3                Dan Lind  5067766158705439  4000000    COP
4          Alexis Roberts  5073370466100227  5000000    COP
5  Darlene Breitenberg MD  5002317704922884  6000000    COP
6          Eduardo Raynor  5034447474697952  7000000    COP
7                Guy Bode  5000503914057304  7000000    COP
8          Freddie Cronin  5089854966724778  8000000    COP
9   Dolores McDermott Jr.  5049890096384389  9000000    COP


In [59]:
pan_list = df_cinta["pan"].to_list() 

print(pan_list)

[5053469667779484, 5086090068617676, 5054876152300379, 5067766158705439, 5073370466100227, 5002317704922884, 5034447474697952, 5000503914057304, 5089854966724778, 5049890096384389]


In [89]:
def encrypt_pans(pan_list,key,roundnum):
    enc_pan = []
    for i in pan_list: 
        enc_pan.append(int(mainloop(i, key, roundnum, False))) 
    return enc_pan

def decrypt_pans(pan_list,key,roundnum):
    dec_pan = []
    for i in pan_list: 
        dec_pan.append(int(mainloop(i, key, roundnum, True))) 
    return dec_pan

In [90]:
key = 32*"A"
roundnum = 6 

encripted = encrypt_pans(pan_list,key,roundnum) 

print(encripted)

[7831625374296051, 9649487490842081, 8568017274320611, 4213627223348113, 9936852479993915, 6255437068359699, 5187423474681170, 1909876531237472, 1984690935045656, 6690829505878464]


In [91]:
key = 32*"A"
roundnum = 6 

decripted = decrypt_pans(encripted,key,roundnum) 

print(decripted)

[5053469667779484, 5086090068617676, 5054876152300379, 5067766158705439, 5073370466100227, 5002317704922884, 5034447474697952, 5000503914057304, 5089854966724778, 5049890096384389]


In [96]:
df_cinta["pan"] = encripted

In [97]:
df_cinta

,nombre,pan,valor,moneda
0,Stella Howell Jr.,7831625374296051,1000000,COP
1,Tracy Schaefer,9649487490842081,2000000,COP
2,Joel Jones,8568017274320611,3000000,COP
3,Dan Lind,4213627223348113,4000000,COP
4,Alexis Roberts,9936852479993915,5000000,COP
5,Darlene Breitenberg MD,6255437068359699,6000000,COP
6,Eduardo Raynor,5187423474681170,7000000,COP
7,Guy Bode,1909876531237472,7000000,COP
8,Freddie Cronin,1984690935045656,8000000,COP
9,Dolores McDermott Jr.,6690829505878464,9000000,COP


In [98]:
facturas = df_cinta.to_dict('records')
print(facturas)

[{'nombre ': 'Stella Howell Jr.', 'pan': 7831625374296051, 'valor': 1000000, 'moneda': 'COP'}, {'nombre ': 'Tracy Schaefer', 'pan': 9649487490842081, 'valor': 2000000, 'moneda': 'COP'}, {'nombre ': 'Joel Jones', 'pan': 8568017274320611, 'valor': 3000000, 'moneda': 'COP'}, {'nombre ': 'Dan Lind', 'pan': 4213627223348113, 'valor': 4000000, 'moneda': 'COP'}, {'nombre ': 'Alexis Roberts', 'pan': 9936852479993915, 'valor': 5000000, 'moneda': 'COP'}, {'nombre ': 'Darlene Breitenberg MD', 'pan': 6255437068359699, 'valor': 6000000, 'moneda': 'COP'}, {'nombre ': 'Eduardo Raynor', 'pan': 5187423474681170, 'valor': 7000000, 'moneda': 'COP'}, {'nombre ': 'Guy Bode', 'pan': 1909876531237472, 'valor': 7000000, 'moneda': 'COP'}, {'nombre ': 'Freddie Cronin', 'pan': 1984690935045656, 'valor': 8000000, 'moneda': 'COP'}, {'nombre ': 'Dolores McDermott Jr.', 'pan': 6690829505878464, 'valor': 9000000, 'moneda': 'COP'}]


In [93]:
# Es necesario tener abierto el Daemon de Mongo

import pymongo
from pymongo  import MongoClient 

#127.0.0.1 = localhost
mongoClient =  MongoClient('127.0.0.1', port = 27017) 

print(mongoClient.list_database_names())

['admin', 'config', 'local', 'prueba']


In [99]:

#Creacion de la base de datos 

miDb = mongoClient.data_refund
#Creacion de la coleccion
miCol = miDb.bill_card


x = miCol.insert_many(facturas)
print(x.inserted_ids)

[ObjectId('647d41f7ca7ed3fdf9de9c1e'), ObjectId('647d41f7ca7ed3fdf9de9c1f'), ObjectId('647d41f7ca7ed3fdf9de9c20'), ObjectId('647d41f7ca7ed3fdf9de9c21'), ObjectId('647d41f7ca7ed3fdf9de9c22'), ObjectId('647d41f7ca7ed3fdf9de9c23'), ObjectId('647d41f7ca7ed3fdf9de9c24'), ObjectId('647d41f7ca7ed3fdf9de9c25'), ObjectId('647d41f7ca7ed3fdf9de9c26'), ObjectId('647d41f7ca7ed3fdf9de9c27')]


In [103]:
cursor = miCol.find({})

for document in cursor:
    print(document)

{'_id': ObjectId('647d41f7ca7ed3fdf9de9c1e'), 'nombre ': 'Stella Howell Jr.', 'pan': 7831625374296051, 'valor': 1000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c1f'), 'nombre ': 'Tracy Schaefer', 'pan': 9649487490842081, 'valor': 2000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c20'), 'nombre ': 'Joel Jones', 'pan': 8568017274320611, 'valor': 3000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c21'), 'nombre ': 'Dan Lind', 'pan': 4213627223348113, 'valor': 4000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c22'), 'nombre ': 'Alexis Roberts', 'pan': 9936852479993915, 'valor': 5000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c23'), 'nombre ': 'Darlene Breitenberg MD', 'pan': 6255437068359699, 'valor': 6000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c24'), 'nombre ': 'Eduardo Raynor', 'pan': 5187423474681170, 'valor': 7000000, 'moneda': 'COP'}
{'_id': ObjectId('647d41f7ca7ed3fdf9de9c25'), 'nombre 